<a href="https://colab.research.google.com/github/wesslen/seamless_sacrebleu_evaluation/blob/main/notebooks/01_chunking_alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import spacy
from pathlib import Path
from typing import List, Dict, Tuple, Optional
import PyPDF2
import re
from dataclasses import dataclass
from collections import defaultdict
import math

@dataclass
class ChunkPair:
    source: str
    target: str
    chunk_type: str  # 'sentence' or 'recursive'
    alignment_score: float = 0.0

class DocumentReader:
    @staticmethod
    def read_text(file_path: str) -> str:
        """Read content from a text file."""
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()

    @staticmethod
    def read_pdf(file_path: str) -> str:
        """Read content from a PDF file."""
        text = []
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text.append(page.extract_text())
        return '\n'.join(text)

    @staticmethod
    def read_document(file_path: str) -> str:
        """Read content from either text or pdf file."""
        suffix = Path(file_path).suffix.lower()
        if suffix in ['.txt', '.md']:
            return DocumentReader.read_text(file_path)
        elif suffix == '.pdf':
            return DocumentReader.read_pdf(file_path)
        else:
            raise ValueError(f"Unsupported file format: {suffix}")

class SentenceTokenizer:
    def __init__(self, language: str):
        """Initialize tokenizer for a specific language."""
        self.language = language
        # Create blank model
        self.nlp = spacy.blank(language)
        # Add sentencizer component
        self.nlp.add_pipe("sentencizer")

        # Common abbreviations by language
        self.abbreviations = {
            'en': {'Mr.', 'Mrs.', 'Dr.', 'Ms.', 'Prof.', 'Sr.', 'Jr.', 'etc.'},
            'es': {'Sr.', 'Sra.', 'Dr.', 'Dra.', 'Prof.', 'etc.'},
            'fr': {'M.', 'Mme.', 'Dr.', 'Prof.', 'etc.'}
        }

        # Sentence ending punctuation by language
        self.sent_endings = {
            'en': {'!', '?', '.', ';"'},
            'es': {'!', '¡', '?', '¿', '.', ';'},
            'fr': {'!', '?', '.', ';'}
        }

    def protect_abbreviations(self, text: str) -> str:
        """Replace periods in abbreviations with a special marker."""
        protected_text = text
        for abbr in self.abbreviations.get(self.language, set()):
            protected_text = re.sub(
                r'\b' + re.escape(abbr) + r'\b',
                lambda m: m.group().replace('.', '@POINT@'),
                protected_text
            )
        return protected_text

    def restore_abbreviations(self, text: str) -> str:
        """Restore the original periods in abbreviations."""
        return text.replace('@POINT@', '.')

    def clean_text(self, text: str) -> str:
        """Clean text while preserving important whitespace and structure."""
        # Replace multiple newlines with single newline
        text = re.sub(r'\n\s*\n', '\n', text)
        # Replace multiple spaces with single space
        text = re.sub(r' +', ' ', text)
        # Clean up whitespace around punctuation
        text = re.sub(r'\s+([.,!?;])', r'\1', text)
        return text.strip()

    def tokenize(self, text: str) -> List[str]:
        """Split text into sentences."""
        # Clean and protect text
        text = self.clean_text(text)
        text = self.protect_abbreviations(text)

        # Process with spaCy
        doc = self.nlp(text)
        sentences = []

        for sent in doc.sents:
            sent_text = str(sent).strip()
            # Only add if it ends with sentence-ending punctuation
            if sent_text and sent_text[-1] in self.sent_endings.get(self.language, {'.', '!', '?'}):
                sentences.append(self.restore_abbreviations(sent_text))

        return sentences

class RecursiveChunker:
    def __init__(self, min_chunk_size: int = 100, max_chunk_size: int = 500):
        self.min_chunk_size = min_chunk_size
        self.max_chunk_size = max_chunk_size

    def chunk_text(self, sentences: List[str]) -> List[str]:
        """Recursively chunk text while preserving sentence boundaries."""
        chunks = []
        current_chunk = []
        current_length = 0

        for sentence in sentences:
            sentence_length = len(sentence)

            # If single sentence exceeds max_chunk_size, keep it as a separate chunk
            if sentence_length > self.max_chunk_size:
                if current_chunk:
                    chunks.append(' '.join(current_chunk))
                    current_chunk = []
                    current_length = 0
                chunks.append(sentence)
                continue

            # If adding this sentence would exceed max_chunk_size
            if current_length + sentence_length > self.max_chunk_size and current_chunk:
                chunks.append(' '.join(current_chunk))
                current_chunk = []
                current_length = 0

            current_chunk.append(sentence)
            current_length += sentence_length

        # Add remaining chunk if it exists
        if current_chunk:
            chunks.append(' '.join(current_chunk))

        return chunks

class TextStatistics:
    @staticmethod
    def calculate_similarity_score(source: str, target: str) -> float:
        """Calculate a similarity score based on various text statistics."""
        # Length ratio
        len_ratio = len(target) / len(source) if len(source) > 0 else float('inf')

        # Sentence count ratio
        source_sent_count = len(re.findall(r'[.!?]+', source))
        target_sent_count = len(re.findall(r'[.!?]+', target))
        sent_ratio = target_sent_count / source_sent_count if source_sent_count > 0 else float('inf')

        # Normalize ratios to 0-1 range
        len_score = max(0, 1 - abs(1 - len_ratio))
        sent_score = max(0, 1 - abs(1 - sent_ratio))

        # Combine scores
        return (len_score + sent_score) / 2

class DocumentAligner:
    def __init__(self, source_lang: str = 'es', target_lang: str = 'en'):
        self.source_tokenizer = SentenceTokenizer(source_lang)
        self.target_tokenizer = SentenceTokenizer(target_lang)
        self.chunker = RecursiveChunker()

    def verify_alignment(self, source: str, target: str) -> Tuple[bool, float]:
        """Verify if two chunks are likely to be aligned."""
        similarity_score = TextStatistics.calculate_similarity_score(source, target)
        is_aligned = similarity_score >= 0.7  # threshold for alignment
        return is_aligned, similarity_score

    def align_chunks(self, source_chunks: List[str], target_chunks: List[str]) -> List[ChunkPair]:
        """Align chunks based on similarity scores and position."""
        aligned_pairs = []

        if len(source_chunks) != len(target_chunks):
            print(f"Warning: Number of chunks differs - Source: {len(source_chunks)}, Target: {len(target_chunks)}")

        for i, source_chunk in enumerate(source_chunks):
            if i >= len(target_chunks):
                print(f"Warning: Missing target chunk for source chunk {i+1}")
                continue

            target_chunk = target_chunks[i]
            is_aligned, similarity_score = self.verify_alignment(source_chunk, target_chunk)

            chunk_type = 'sentence' if len(source_chunk.split()) <= 2 else 'recursive'

            if not is_aligned:
                print(f"\nWarning: Possible misalignment in chunk {i+1}")
                print(f"Source: {source_chunk[:100]}...")
                print(f"Target: {target_chunk[:100]}...")
                chunk_type = 'unverified'

            aligned_pairs.append(ChunkPair(
                source=source_chunk,
                target=target_chunk,
                chunk_type=chunk_type,
                alignment_score=similarity_score
            ))

        return aligned_pairs

    def process_documents(self, source_path: str, target_path: str) -> List[ChunkPair]:
        """Process and align documents."""
        print(f"\nProcessing documents:")
        print(f"Source: {source_path}")
        print(f"Target: {target_path}")

        # Read documents
        source_text = DocumentReader.read_document(source_path)
        target_text = DocumentReader.read_document(target_path)

        print("Tokenizing sentences...")
        # Get sentences
        source_sentences = self.source_tokenizer.tokenize(source_text)
        target_sentences = self.target_tokenizer.tokenize(target_text)

        print(f"Found {len(source_sentences)} source sentences and {len(target_sentences)} target sentences")

        print("Creating chunks...")
        # Create chunks
        source_chunks = self.chunker.chunk_text(source_sentences)
        target_chunks = self.chunker.chunk_text(target_sentences)

        print(f"Created {len(source_chunks)} source chunks and {len(target_chunks)} target chunks")

        print("Aligning chunks...")
        # Align chunks
        return self.align_chunks(source_chunks, target_chunks)

def process_alignment_file(input_path: str, output_path: str):
    """Process a JSONL file containing document pairs and create aligned chunks."""
    aligner = DocumentAligner()

    # Create output directory if it doesn't exist
    output_dir = Path(output_path).parent
    output_dir.mkdir(parents=True, exist_ok=True)

    aligned_pairs = []

    # Read input file
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            pair = json.loads(line)
            source_path = pair['source']
            target_path = pair['target']

            # Process document pair
            chunk_pairs = aligner.process_documents(source_path, target_path)

            # Convert to required output format
            for pair in chunk_pairs:
                if pair.alignment_score >= 0.7:  # Only include high-confidence alignments
                    aligned_pairs.append({
                        "source_text": pair.source,
                        "references": [pair.target],
                        "chunk_type": pair.chunk_type,
                        "alignment_score": pair.alignment_score
                    })

    # Write output file
    with open(output_path, 'w', encoding='utf-8') as f:
        for pair in aligned_pairs:
            json.dump(pair, f, ensure_ascii=False)
            f.write('\n')

    print(f"\nProcessing complete!")
    print(f"Total aligned pairs: {len(aligned_pairs)}")
    print(f"Output written to: {output_path}")

# Example usage in Jupyter notebook
if __name__ == "__main__":
    # Replace with your actual file paths
    input_file = "document_pairs.jsonl"
    output_file = "aligned_chunks.jsonl"

    process_alignment_file(input_file, output_file)